### Função

In [7]:

import os
def read_directories(directory, img=None):
    # Get a list of filenames in the specified directory
    filenames = []
    for filename in os.listdir(directory):
        if img is not None:
            # If 'img' is provided, filter filenames containing it
            if img in filename:   
                filenames.append(filename)          
        else:
            filenames.append(filename)    
    return filenames

### Diretório das imagens

In [9]:

dir_images_atlas = f'work1/atlas'
dir_images_denoising_non_local_means = f'work1/denoising/non_local_means/images_filtered'

dir_out_resampled = f'work1/atlas_resampled'


array_images_atlas = read_directories(dir_images_atlas)
array_images_denoising = read_directories(dir_images_denoising_non_local_means)



In [13]:
teste = f'{dir_out_resampled}/{array_images_denoising[0]}/'
print(teste)

work1/atlas_resampled/IXI002-Guys-0828-T1.nii.gz


In [5]:
import SimpleITK as sitk
import numpy as np

# Função para reamostrar a imagem, ajustando resolução, tamanho e propriedades
def resample_image(itk_image, out_spacing, out_size=None, reference_image=None, is_label=False):
    resample = sitk.ResampleImageFilter()

    # Define o tamanho e o espaçamento
    if reference_image is not None:
        # Usa as propriedades da imagem de referência
        resample.SetSize(reference_image.GetSize())
        resample.SetOutputSpacing(reference_image.GetSpacing())
        resample.SetOutputDirection(reference_image.GetDirection())
        resample.SetOutputOrigin(reference_image.GetOrigin())
    else:
        original_spacing = itk_image.GetSpacing()
        original_size = itk_image.GetSize()
        if out_size is None:
            # Calcula o novo tamanho baseado no espaçamento
            out_size = [
                int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
                int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
                int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))
            ]
        resample.SetSize(out_size)
        resample.SetOutputSpacing(out_spacing)
        resample.SetOutputDirection(itk_image.GetDirection())
        resample.SetOutputOrigin(itk_image.GetOrigin())

    # Configurações gerais do filtro
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    # Escolhe o interpolador com base no tipo da imagem
    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)


# Programa principal
reader = sitk.ImageFileReader()
reader.SetFileName('data\\IXI002-Guys-0828-T1.nii.gz')
img = reader.Execute()

reader = sitk.ImageFileReader()
reader.SetFileName('atlas\\mni_csf.nii.gz')
img_atlas = reader.Execute()

print("Informação imagem original:")
print(f"Tamanho: {img.GetSize()}")
print(f"Espaçamento: {img.GetSpacing()}")
print(f"Origem: {img.GetOrigin()}")
print(f"Direção: {img.GetDirection()}")

print("Informação do atlas:")
print(f"Tamanho: {img_atlas.GetSize()}")
print(f"Espaçamento: {img_atlas.GetSpacing()}")
print(f"Origem: {img_atlas.GetOrigin()}")
print(f"Direção: {img_atlas.GetDirection()}")


# Reamostra o atlas para coincidir com a imagem original
img_resampled = resample_image(img_atlas, out_spacing=img.GetSpacing(), reference_image=img)

print("Informação imagem reamostrada:")
print(f"Tamanho: {img_resampled.GetSize()}")
print(f"Espaçamento: {img_resampled.GetSpacing()}")
print(f"Origem: {img_resampled.GetOrigin()}")
print(f"Direção: {img_resampled.GetDirection()}")


# Salva a imagem reamostrada
writer = sitk.ImageFileWriter()
writer.SetFileName("atlas_resampled_to_img.nii.gz")
writer.Execute(img_resampled)


RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: D:\a\SimpleITK\SimpleITK\Code\IO\src\sitkImageReaderBase.cxx:91:
sitk::ERROR: The file "data\IXI002-Guys-0828-T1.nii.gz" does not exist.